In [34]:
import numpy as np
from fractions import Fraction
import scipy as sp

## Calculating electronic energy

This notebook will focus on calculating electronic energy, which is denoted by the Hamiltonian shown below.
#### $$\hat{H} = -\sum_{i} \sum_{j} \frac{Z_{i}e^{2}}{4\pi\epsilon_{0}|R_{i} - r{j}|} + \sum_{i} \sum_{j > i} \frac{e^{2}}{4\pi\epsilon_{0}|r_{i} - r_{j}|}$$
As you can see, we're ignoring three terms within the typical Coulombic molecular Hamiltonian. Namely, the kinetic energy terms for the nucleus and the electrons, as well as the potential energy term for the Coulombic nuclei-nuclei repulsions, or eletric potential energy. The reason for that is because the kinetic energy terms require a $\psi$ term, which is almost impossible to derive. The term below is the simplified Hamiltonian we're working with and is also the "objective function" we're trying to minimize for. 
#### $$\hat{H} = -\sum_{i = 1}^{N} \frac{3e^{2}}{4\pi\epsilon_{0}r_{i}} + \sum_{i \neq j}^{N} \frac{e^{2}}{4\pi\epsilon_{0}r_{i, j}}$$

## Constraints
The constraints are that $r_{i}$ must be greater than 0 and that the $x$ and $y$ values of $r_{1}$ and $r_{2}$ must add up to the $x$ and $y$ value of $r_{1, 2}$. 
#### $$\sum_{i} r_{i} > 0\ \&\ \sum_{i} \sum_{j} \|r_{i}\| + \|r_{j}\| = r_{i, j}$$

## Constructing a molecule

For this notebook, we will be looking exclusively at 1:1 ratios of proton/neutrons and electrons. This simplifies up the molecular Hamiltonian to an easily reducible form. 

In [28]:
def construct_molecules(size : int):
	terms = {}
	terms['N'] = [np.float64(0), np.float64(0)]
	parts = (2 * np.pi) / size
	for i in range(size):
		phi = parts * i
		terms['e' + str(i + 1)] = []
		terms['e' + str(i + 1)].append(np.cos(phi))
		terms['e' + str(i + 1)].append(np.sin(phi))
	return terms

In [29]:
atom = construct_molecules(size=2)

In [30]:
atom

{'N': [np.float64(0.0), np.float64(0.0)],
 'e1': [np.float64(1.0), np.float64(0.0)],
 'e2': [np.float64(-1.0), np.float64(1.2246467991473532e-16)]}

In [67]:
def construct_hamiltonian(atom):
  terms = []
  coefficients = []
  for i in range(len(atom) - 1):
    x_dist = atom['e' + str(i + 1)][0] - atom['N'][0]
    y_dist = atom['e' + str(i + 1)][1] - atom['N'][1]
    terms.append([x_dist, y_dist])
    coefficients.append(4 * np.pi * sp.constants.epsilon_0 * np.sqrt((x_dist ** 2) + (y_dist ** 2)))
  if (len(atom) - 1 == 2):
    x_dist = atom['e2'][0] - atom['e1'][0]
    y_dist = atom['e2'][1] - atom['e1'][1]
    terms.append([x_dist, y_dist])
    coefficients.append(4 * np.pi * sp.constants.epsilon_0 * np.sqrt((x_dist ** 2) + (y_dist ** 2)))
  else:
    for i in range(len(atom) - 1):
      x_dist = atom['e' + str(3 % (i + 2) + 1)][0] - atom['e' + str(3 % (i + 1) + 1)][0]
      y_dist = atom['e' + str(3 % (i + 2) + 1)][1] - atom['e' + str(3 % (i + 1) + 1)][1]
      terms.append([x_dist, y_dist])
      coefficients.append(4 * np.pi * sp.constants.epsilon_0 * np.sqrt((x_dist ** 2) + (y_dist ** 2)))
  return terms, coefficients

In [68]:
atom_term, atom_coefficent = construct_hamiltonian(atom=atom)

In [69]:
atom_term

[[np.float64(1.0), np.float64(0.0)],
 [np.float64(-1.0), np.float64(1.2246467991473532e-16)],
 [np.float64(-2.0), np.float64(1.2246467991473532e-16)]]

In [70]:
atom_coefficent

[np.float64(1.1126500562018527e-10),
 np.float64(1.1126500562018527e-10),
 np.float64(2.2253001124037054e-10)]